In [ ]:
import numpy as np
import pandas as pd

import lightgbm as lgb
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import (SimpleImputer,KNNImputer, IterativeImputer)
from sklearn.ensemble import (RandomForestRegressor, IsolationForest)
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures, normalize
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import ExtraTreesRegressor

from sklearn.metrics import r2_score

import matplotlib.pyplot as plt
from tqdm import tqdm
%matplotlib inline

In [ ]:
def normalize_data(X):
    # normalization
    X_std = np.nanstd(X,axis=0,keepdims=True)
    X_ave = np.nanmean(X,axis=0,keepdims=True)
    X_norma = (X-X_ave)/X_std
    return X_norma

def fill_missing_values(X, n_neighbors = 75, method="KNN"): 
    imputer = KNNImputer(missing_values=np.nan, n_neighbors=n_neighbors, weights = 'distance') if method=="KNN"\
        else IterativeImputer(random_state=0)
    X_fixed = imputer.fit_transform(X)
    return X_fixed

def remove_outliers(X, y):
    print("IsolationForest-Traing data shape before removed: {}".format(X.shape))
    Z = np.c_[X, y]
    iforest = IsolationForest(max_samples=200, random_state=1, contamination=0.005)
    iforest.fit(Z)
    iforest_outlier_pred = iforest.predict(Z)
    
    Z = np.c_[X, y]
    local = LocalOutlierFactor(n_neighbors=150, contamination=0.005)
    local.fit(Z)
    local_outlier_pred = local.fit_predict(Z)

    mask = np.logical_and((iforest_outlier_pred!=-1), (local_outlier_pred!=-1))
    return mask

def find_missing_value_and_move_outliers(X, y, X_test):
    print("Traing data shape before impute and outlier remove: {}".format(X.shape))
    print("Testing data shape before impute and outlier remove: {}".format(X_test.shape))
    X_nan = np.isnan(X)  
    X_median = fill_missing_values(X, method="median")
    
    IstInLiers = remove_outliers(X_median, y)
    X_median = X_median[IstInLiers, :]
    X_nan = X_nan[IstInLiers, :]
    y = y[IstInLiers]
    X_median[X_nan] = np.nan
    
    X_median = normalize_data(X_median)
    X_test = normalize_data(X_test)
    
#     X_all = np.r_[X_median, X_test]
    X_KNN = fill_missing_values(X_median, method="KNN")
    X_test_KNN = fill_missing_values(X_test, method="KNN")
    
#     X_all_KNN = fill_missing_values(X_all, method="KNN")
#     X_KNN = X_all_KNN[:X_median.shape[0]]
#     X_test_KNN = X_all_KNN[X_median.shape[0]:]
    print("Traing data shape after impute and outlier remove: {}".format(X_KNN.shape))
    print("Testing data shape after impute and outlier remove: {}".format(X_test_KNN.shape))
    
    return X_KNN, X_test_KNN, y

def select_features(X, y, X_test, feature_num=50):
    rf = RandomForestRegressor(n_jobs=-1, n_estimators=80, random_state=1)
    rf.fit(X, y)
    indices = np.asarray(list(rf.feature_importances_)).argsort()[-feature_num:][::-1]
    
    X = np.take(X, indices, axis = 1)
    X_test = np.take(X_test, indices, axis = 1)
    return X, X_test

In [ ]:
X_train_data = pd.read_csv('X_train.csv')
y_train_data = pd.read_csv('y_train.csv')
X_test_data = pd.read_csv('X_test.csv')

indices_test = np.array(X_test_data)[:,0]
X_test = np.array(X_test_data)[:,1:]
y_train = np.array(y_train_data)[:,1]
X_train = np.array(X_train_data)[:,1:]

## 1. Normalization
* There are four fucking columns containing constant values.

In [ ]:
# X_train = normalize_data(X_train)
# X_test = normalize_data(X_test)

## 2. Imputation of Missing Values & Outlier detection
* Imputation [Reference](https://scikit-learn.org/stable/modules/impute.html)
* Outlier [reference_sklearn](https://scikit-learn.org/stable/modules/outlier_detection.html)
* Outlier [reference_in_detail](https://practicaldatascience.co.uk/machine-learning/how-to-use-the-isolation-forest-model-for-outlier-detection)
* We use median first and KNN again after outlier detection

In [ ]:
X_train, X_test, y_train = find_missing_value_and_move_outliers(X_train, y_train, X_test)

## 3. Feature Selection

In [ ]:
# X, X_test = feature_reduction(X, X_test,750)
X_train, X_test = select_features(X_train, y_train, X_test,feature_num = 50)
print("Traing data shape after selection: {}".format(X_train.shape))
print("Testing data shape after selection: {}".format(X_test.shape))

## 4. Extra Tree

In [ ]:
def fit_model_and_pred(X_train, y_train, X_val, y_val, X_test, random_state=0):
    model = ExtraTreesRegressor(n_jobs=1, max_depth=None, n_estimators=190, random_state=random_state, min_samples_split=3, max_features=None)
    model.fit(X_train, y_train)
    
    y_val_pred = model.predict(X_val)
    score = r2_score(y_val, y_val_pred)
    
    y_pred = model.predict(X_test) 
    
    return score, y_pred

def train_k_fold(X, y, fold_num=10):
    kf = KFold(n_splits=fold_num, shuffle=False)
    kf.get_n_splits(X)
    test_score = 0.0
    
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_val, random_state=0)

        print('The obtained validation r2 score is : ',score)
        test_score += score
    print("Validation score: %f"%(test_score/fold_num))
    
def train_k_fold_predict(X, y, X_test, fold_num=10, random_state=0):
    kf = KFold(n_splits=fold_num)
    kf.get_n_splits(X)
    y_test_predict = np.zeros(X_test.shape[0])
    cnt = 0
    val_score = 0.0
        
    for train_index, test_index in kf.split(X):
        X_train, X_val = X[train_index], X[test_index]
        y_train, y_val = y[train_index], y[test_index]

        score, y_pred = fit_model_and_pred(X_train, y_train, X_val, y_val, X_test, random_state=random_state)
        val_score += score
        if(score > 0.68):
            y_test_predict += y_pred
            cnt += 1
    return val_score/fold_num, y_test_predict/cnt

def train_random_iterations(X, y, X_test, iterations=100):
    y_test_predict = np.zeros(X_test.shape[0])
    y_pred_best = np.zeros(X_test.shape[0])
    cnt = 0
    best_score = -10000
    
    for i in tqdm(range(iterations)):
        score, y_pred = train_k_fold_predict(X, y, X_test, random_state=i)
        if(score > 0.635):
            print(score)
            y_pred_best += y_pred
            cnt += 1
            
    return y_pred_best / cnt
    

In [ ]:
train_k_fold(X_train,y_train) #Knn with std and feature num = 100

In [ ]:
_, y_test_pred = train_k_fold_predict(X_train, y_train, X_test)
final_res = np.vstack((indices_test, y_test_pred)).T

In [ ]:
pd.DataFrame(final_res).to_csv("our_result.csv", header = ["id", "y"], index=False)